In [23]:
import pandas as pd
import numpy as np

# Load daily stock prices
df = pd.read_csv("C:/Users/user/Downloads/daily_prices_hist_5y.csv")

# Convert date column to datetime
df["trade_date"] = pd.to_datetime(df["trade_date"])

# Sort properly (very important for time series)
df = df.sort_values(["ticker", "trade_date"])

# Safety check to avoid KeyError
if df.index.name == "trade_date":
    df = df.reset_index()

In [24]:
def fill_missing_business_days(group):
    # Create continuous business-day index
    full_index = pd.date_range(
        start=group["trade_date"].min(),
        end=group["trade_date"].max(),
        freq="B"
    )

    # Reindex safely
    group = (
        group.set_index("trade_date")
             .reindex(full_index)
    )

    # Restore ticker
    group["ticker"] = group["ticker"].iloc[0]

    return group.reset_index(names="trade_date")


df = (
    df.groupby("ticker", group_keys=False)
      .apply(fill_missing_business_days)
)

C:\Users\user\AppData\Local\Temp\ipykernel_22960\171233182.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_missing_business_days)


In [25]:
price_cols = ["open_price", "high_price", "low_price", "close_price"]

df[price_cols] = (
    df.groupby("ticker")[price_cols]
      .ffill()
)

In [26]:
df["daily_return"] = (
    df.groupby("ticker")["close_price"]
      .pct_change()
)

In [27]:
df["ma_20"] = (
    df.groupby("ticker")["close_price"]
      .transform(lambda x: x.rolling(20).mean())
)

df["ma_50"] = (
    df.groupby("ticker")["close_price"]
      .transform(lambda x: x.rolling(50).mean())
)

In [28]:
df["volatility_30d"] = (
    df.groupby("ticker")["daily_return"]
      .transform(lambda x: x.rolling(30).std())
)

In [29]:
df["cumulative_return"] = (
    df.groupby("ticker")["daily_return"]
      .transform(lambda x: (1 + x).cumprod() - 1)
)

In [30]:
df["price_direction"] = np.where(
    df["daily_return"] > 0, "UP",
    np.where(df["daily_return"] < 0, "DOWN", "FLAT")
)

In [31]:
df["rolling_max"] = (
    df.groupby("ticker")["close_price"]
      .cummax()
)

df["drawdown"] = (
    df["close_price"] - df["rolling_max"]
) / df["rolling_max"]


In [32]:
monthly_summary = (
    df.groupby("ticker")
      .resample("M", on="trade_date")
      .agg(
          avg_close_price=("close_price", "mean"),
          total_volume=("volume", "sum"),
          avg_return=("daily_return", "mean")
      )
      .reset_index()
)


C:\Users\user\AppData\Local\Temp\ipykernel_22960\1171598535.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M", on="trade_date")


In [33]:
final_df = df[[
    "trade_date",
    "ticker",
    "open_price",
    "high_price",
    "low_price",
    "close_price",
    "daily_return",
    "ma_20",
    "ma_50",
    "volatility_30d",
    "cumulative_return",
    "price_direction",
    "drawdown",
    "volume"
]]

final_df.head()


,trade_date,ticker,open_price,high_price,low_price,close_price,daily_return,ma_20,ma_50,volatility_30d,cumulative_return,price_direction,drawdown,volume
0,2020-12-25,AAPL,1829.50,1845.45,1779.32,1799.24,NaN,NaN,NaN,NaN,NaN,FLAT,0.000000,528162
1,2020-12-28,AAPL,1798.51,1810.18,1787.49,1797.92,-0.000734,NaN,NaN,NaN,-0.000734,DOWN,-0.000734,1314072
2,2020-12-29,AAPL,1803.92,1807.23,1778.71,1790.47,-0.004144,NaN,NaN,NaN,-0.004874,DOWN,-0.004874,289760
3,2020-12-30,AAPL,1790.93,1825.89,1773.23,1810.59,0.011237,NaN,NaN,NaN,0.006308,UP,0.000000,216247
4,2020-12-31,AAPL,1813.22,1846.98,1793.07,1829.50,0.010444,NaN,NaN,NaN,0.016818,UP,0.000000,241459


In [34]:
display(final_df)

,trade_date,ticker,open_price,high_price,low_price,close_price,daily_return,ma_20,ma_50,volatility_30d,cumulative_return,price_direction,drawdown,volume
0,2020-12-25,AAPL,1829.50,1845.45,1779.32,1799.24,NaN,NaN,NaN,NaN,NaN,FLAT,0.000000,528162
1,2020-12-28,AAPL,1798.51,1810.18,1787.49,1797.92,-0.000734,NaN,NaN,NaN,-0.000734,DOWN,-0.000734,1314072
2,2020-12-29,AAPL,1803.92,1807.23,1778.71,1790.47,-0.004144,NaN,NaN,NaN,-0.004874,DOWN,-0.004874,289760
3,2020-12-30,AAPL,1790.93,1825.89,1773.23,1810.59,0.011237,NaN,NaN,NaN,0.006308,UP,0.000000,216247
4,2020-12-31,AAPL,1813.22,1846.98,1793.07,1829.50,0.010444,NaN,NaN,NaN,0.016818,UP,0.000000,241459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,2025-12-19,TSLA,861.36,880.48,850.02,879.15,0.022208,899.4940,899.1440,0.020706,1.365723,UP,-0.241216,310691
1301,2025-12-22,TSLA,878.43,884.70,871.75,883.64,0.005107,896.2480,899.2752,0.020722,1.377805,UP,-0.237341,486410
1302,2025-12-23,TSLA,882.57,888.31,864.43,874.46,-0.010389,894.6030,899.2762,0.020725,1.353103,DOWN,-0.245264,325221
1303,2025-12-24,TSLA,875.39,884.01,864.13,866.14,-0.009514,892.5645,899.4788,0.020503,1.330714,DOWN,-0.252445,1019002
